# Code to generate the arrow plots!

Testing to get the plots just right. I'll load in a single file, and make some of the subfunctions to do this

In [32]:
from dna_features_viewer import GraphicFeature, GraphicRecord
import bokeh
from bokeh.plotting import figure, output_file, save, show
from bokeh.io import output_notebook
import matplotlib.pyplot as plt
import matplotlib.font_manager
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import pandas as pd
import os
import pdb
from bokeh.embed import file_html
from bokeh.resources import CDN

SyntaxError: invalid syntax (2618925454.py, line 1)

In [16]:
gffDF.iloc[0:5,8]

0    ID=gene-PMAA_034170;Name=PMAA_034170;end_range...
1    ID=rna-PMAA_034170;Parent=gene-PMAA_034170;Not...
2    ID=exon-PMAA_034170-1;Parent=rna-PMAA_034170;N...
3    ID=cds-EEA28616.1;Parent=rna-PMAA_034170;Dbxre...
4    ID=gene-PMAA_034180;Name=PMAA_034180;gbkey=Gen...
Name: info, dtype: object

In [18]:
gffBGC = r'/Users/gnickles/Desktop/FungalICS_Website/Data/ICSBGCs/GCA_000001985.1/Cluster_1/1_Talaromyces_marneffei.gff'
domains = r'./Data/SupTable4.tsv'
domainsDF = pd.read_csv(domains, sep='\t')

gffDF = pd.read_csv(gffBGC, sep='\t', names = ['contig','source','featureType','start','stop','n1','dir','n2','info'])


#subfunction that defines the color of the gene arrows
def GetColor(domains):
    colorDictionary = {"MultiDomain": "#F94144",
    "ICS": "#90BE6D",
    "MFS": "#F3722C",
    "p450": "#F8961E",
    "Isocyanide hydratase": "#43AA8B",
    "No domain": "#577590",
    "Other": "#F9C74F"}

    #Rules, if there is an ICS, always color that color. Otherwise if it's multidomain, default to that 
    #no domain
    if len(domains) == 0:
        return colorDictionary["No domain"]
    #ics
    if "DIT1_PvcA" in domains:
        return colorDictionary["ICS"]
    #multidomain
    if len(domains) > 1:
        return colorDictionary['MultiDomain']
    #other specified domains
    if "MFS" in domains:
        return colorDictionary['MFS']
    elif "p450" in domains:
        return colorDictionary['p450']
    elif "DJ-1_PfpI" in domains:
        return colorDictionary["Isocyanide hydratase"]
    else:
        return colorDictionary['Other']


#subfunction to get the names and proteins
def CreateGeneMapping(gffDF):
    bgcMapping = {}
    genesOnly = gffDF[gffDF['featureType'] == "gene"]
    for index, row in genesOnly.iterrows():
        #gathering all of the information
        geneStart = row['start']
        geneStop = row['stop']
        geneName = row['info'].split(";")[0].split("gene-")[-1]
        direction = row['dir']
        cdsRegions = gffDF[gffDF['info'].str.contains(geneName)]
        if len(cdsRegions) == 0:
            pdb.set_trace()
        topCDS_info = cdsRegions.iloc[0,8]
        proteinName = topCDS_info.split("protein_id=")[-1]
        #getting the domains in this protein
        domainsInProtein = domainsDF[domainsDF['Protein'] == proteinName]
        if len(domainsInProtein) == 0:
            pdb.set_trace()
        domains = domainsInProtein['Domain'].unique().tolist()
        color = GetColor(domains)

        #adding this to the mapping dictionary
        bgcMapping[geneName] = {"start": geneStart,"stop":geneStop,"proteinName":proteinName,"domains":domains, "dir":direction, "color": color}
    #returning the results
    return bgcMapping


#main function to creat the gffDF visualization
def CreateBGC_Vis(gffDF, bgcSequence, bgcName, clusterPath):
    bgcMapping = CreateGeneMapping(gffDF)

    features = []
    for gene in bgcMapping:
        #loading all of the data from the mapping file
        start = bgcMapping['start']
        stop = bgcMapping['stop']
        protName = bgcMapping['proteinName']
        direction_str = bgcMapping["dir"]
        geneColor = bgcMapping["color"]
        if direction_str == "+":
            direction = +1
        else:
            direction = -1

        domains = ", ".join(bgcMapping['domains'])
        features.append(GraphicFeature(start=start, end=stop, strand=direction, color=geneColor, html=domains))
    #making the record object
    seqLength = len(bgcSequence)
    record = GraphicRecord(sequence=bgcSequence, features=features)
    plot = record.plot_with_bokeh(figure_width=8, figure_height="auto")
    #saving this as an html file
    htmlOutput = os.path.join(clusterPath, bgcName + ".html")
    html = file_html(plot, CDN, title=bgcName)
    with open(htmlOutput, "w") as out:
        out.write(html)



#the wrapper function that loops over all the data
def LoopOverBGCs(rootDir):
    for accession in os.listdir(rootDir):
        accPath = os.path.join(rootDir, accession)
        for cluster in os.listdir(accPath):
            clusterPath = os.path.join(accPath, cluster)
            if os.path.isdir(clusterPath) and cluster.startswith("Cluster"):
                #getting the name and file location, this will get passed onto the visualizer program
                clusterName = ""
                gffFile = ""
                for bgcFile in os.listdir(clusterPath):
                    if not bgcFile.startswith("."):
                        if bgcFile.endsWith(".gff"):
                            gffFile = os.path.join(clusterPath, bgcFile)
                        elif bgcFile.endsWith(".gbk"):
                            clusterName = bgcFile.split(".gb")[0]
                        elif bgcFile.endsWith(".fna"):
                            # Open the FASTA file and read the first sequence
                            fnaPath = os.path.join(clusterPath, bgcFile)
                            record = next(SeqIO.parse(fnaPath , "fasta"))
                            # Extract the sequence as a string
                            pdb.set_trace()
                            bgcSequence = str(record.seq)

                #running the visualiztion program
                gffDF = pd.read_csv(gffFile, sep='\t', names = ['contig','source','featureType','start','stop','n1','dir','n2','info'])
                pdb.set_trace()
                CreateBGC_Vis(gffDF, bgcSequence, bgcName, clusterPath)




if __name__ == '__main__':
    rootDir = r'/Users/gnickles/Desktop/FungalICS_Website/Data/ICSBGCs'
    LoopOverBGC(rootDir)



IndentationError: expected an indented block (457761832.py, line 31)